In [4]:
from statsmodels.stats.power import TTestIndPower
import math
from scipy.stats import norm

In [5]:
def ab_test_duration(sigma, delta, alpha=0.05, power=0.8, daily_users=50, alternative='two-sided'):
    """
    Розрахунок тривалості A/B тесту для метрики з нормальним розподілом.
    
    sigma: стандартне відхилення метрики
    delta: мінімальна різниця, яку хочемо виявити
    alpha: рівень значущості (тип I помилка)
    power: потужність тесту (1 - beta)
    daily_users: середня кількість користувачів на день
    alternative: тип альтернативної гіпотези ('two-sided', 'larger', 'smaller')
        'two-sided' — двосторонній тест. Перевіряє, чи є будь-яка різниця між групами (вище чи нижче).
        'larger' — односторонній тест, перевіряє, чи нова група більше за контроль.
        'smaller' — односторонній тест, перевіряє, чи нова група менше за контроль.
    """
    analysis = TTestIndPower()
    
    # effect_size = різниця / стандартне відхилення
    effect_size = delta / sigma
    
    n_per_group = analysis.solve_power(effect_size=effect_size,
                                       alpha=alpha,
                                       power=power,
                                       alternative=alternative)
    
    n_per_group = math.ceil(n_per_group)
    days_needed = math.ceil(n_per_group / daily_users)
    
    return n_per_group, days_needed

In [6]:
sigma = 20          # стандартне відхилення депозиту
delta = 5           # мінімальна різниця, яку хочемо виявити
daily_users = 50    # користувачів на день

n, days = ab_test_duration(sigma, delta, daily_users=daily_users)
print(f"Користувачів на групу: {n}")
print(f"Тривалість тесту (днів): {days}")

Користувачів на групу: 253
Тривалість тесту (днів): 6


Потрібно 253 користувача на кожну групу аби за 6 днів побачити зміни в 5 доларів в середньому депозиті

In [ ]:
def ab_test_proportion(
    p_hat,
    delta,
    alpha=0.05,
    power=0.8,
    daily_users=50,
    alternative='two-sided'
    ):
    """
    Розрахунок кількості користувачів та тривалості A/B тесту для пропорцій.
    p_hat: базова конверсія (наприклад 0.1)
    delta: мінімальний абсолютний ефект (наприклад 0.02)
    alpha: рівень значущості (тип I помилка)
    power: потужність тесту (1 - beta)
    daily_users: середня кількість користувачів на день (на групу)
    alternative: тип гіпотези ('two-sided', 'larger', 'smaller')
    """

    # z_alpha залежить від типу тесту
    if alternative == 'two-sided':
        z_alpha = norm.ppf(1 - alpha / 2)
    else:
        z_alpha = norm.ppf(1 - alpha)

    z_beta = norm.ppf(power)

    # формула для пропорцій
    n_per_group = (2 * (z_alpha + z_beta) ** 2 * p_hat * (1 - p_hat)) / (delta ** 2)

    n_per_group = math.ceil(n_per_group)
    days_needed = math.ceil(n_per_group / daily_users)

    return n_per_group, days_needed

Поточна конверсія (контроль): 5%. Ми хочемо перевірити нову фічу

Бізнес каже - «Має сенс щось робити, якщо конверсія зросте мінімум на 1»

Трафік: 1 000 користувачів на день на групу

Класичні налаштування статистики: α = 0.05, потужність = 80%

In [9]:
n, days = ab_test_proportion(
    p_hat=0.05,
    delta=0.01,
    alpha=0.05,
    power=0.8,
    daily_users=1000,
    alternative='two-sided'
)

print(f"Потрібно користувачів на групу: {n}")
print(f"Тривалість тесту: {days} днів")

Потрібно користувачів на групу: 7457
Тривалість тесту: 8 днів


Вам потрібно 7457 користувачів на групу (контрольну та тестову), щоб виявити покращення на 1% із потужністю 80% і рівнем значущості 5%. Загальний розмір вибірки становить 2⋅7448=14 896 користувачів.